In [113]:
import numpy as np
import os
import random
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [114]:
max_minutiae_count = 70
real_data_path = '/home/chavosh/extracted_minutiaes/real_from_HR/'
generated_data_path = '/home/chavosh/extracted_minutiaes/{}' # 1 to 5

In [115]:
# If I want to use other features:
# Make string features categorical. Also, have a limit of features of a minutiae (and also total number of them)

def extract_features(file_path, max_minutiae_count):
    features = []
    with open(file_path) as f:
        for idx, line in enumerate(f):
            line = line.strip()
            if idx == 0:
                parts = line.split()
                w, h = float(parts[-2]), float(parts[-1])
            if idx >= 4:
                parts = line.replace(';', ',').split(":")
                parts = [part.strip() for part in parts[1:]]
                parts = [[tmp.strip() for tmp in part.split(',')] if part.find(',') >= 0 else part for part in parts]
                tmp = []
                tmp.append(float(parts[0][0]) / w)
                tmp.append(float(parts[0][1]) / h)
                tmp.append(np.sin(float(parts[1]) * np.pi / 16))
                tmp.append(np.cos(float(parts[1]) * np.pi / 16))
                features.append(tmp)
#     random.shuffle(features)
    features = features[:max_minutiae_count]
    result = np.zeros((max_minutiae_count, 4))
    if features:
        result[:len(features), :] = features
    return result

In [116]:
features = []
labels = [] # 0 -> real, 1 -> fake
for minutiae in os.listdir(real_data_path):
    if minutiae[-4:] != '.min':
        continue
    features.append(extract_features(os.path.join(real_data_path, minutiae), max_minutiae_count))
    labels.append(0)
for i in range(1, 6):
    print i
    for minutiae in os.listdir(generated_data_path.format(i)):
        if minutiae[-4:] != '.min':
            continue
        features.append(extract_features(os.path.join(generated_data_path.format(i), minutiae), max_minutiae_count))
        labels.append(1)

1
2
3
4
5


In [117]:
features = np.array(features)
features.shape

(88804, 70, 4)

In [118]:
features = features.reshape((features.shape[0], -1))
features.shape

(88804, 280)

In [119]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)
clf = LinearSVC()
clf.fit(X_train, y_train)
train_acc = accuracy_score(y_train, clf.predict(X_train))
test_acc = accuracy_score(y_test, clf.predict(X_test))

print train_acc, test_acc

0.6641292434274733 0.6517273996666817


#### 20 minutiae -> ~ 0.65
#### 50 minutiae -> ~ 0.65
#### 70 minutiae -> ~ 0.65